In [1]:

import sys
!{sys.executable} -m pip install -r requirements.txt

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [2]:
from tqdm import tqdm
import pandas as pd
import pprint
import project_helper

import lxml

from pymongo import MongoClient


import pymongo  # package for working with MongoDB



from bs4 import BeautifulSoup
import re
import datetime

# import pyspark



import Stocklist



import re
from cPickle import dump
from requests import get


from collections import Counter 


import re, requests









In [3]:
cik_lookup = {
    'AMZN': '1018724',
'BMY': '0000014272',   
    'CNP': '0001130310',
    'CVX': '0000093410',
    'FL': '0000850209',
    'FRT': '0000034903',
    'HON': '0000773840'}

In [4]:
sec_api = project_helper.SecAPI()


In [42]:
%psource project_helper


In [6]:


def get_sec_data(cik, doc_type='',dateb='', owner='include', start=0, count=100,search_text=''):
    newest_pricing_data = pd.to_datetime('2020-07-01')

    
    
    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&dateb={}&owner={}&count={}&search_text={}&output=atom' \
        .format(cik, doc_type, dateb, owner , start ,count , search_text)
    
    print(rss_url)
    
    
    sec_data = sec_api.get(rss_url)
    feed = BeautifulSoup(sec_data.encode('ascii'), 'html').feed
    entries = [
        (
            
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)
        
        if pd.to_datetime(entry.content.find('filing-date').getText()) >= newest_pricing_data]
    print entries

    return entries
    


In [75]:
def get_sec_CIK_From_Ticker(tickers):
    
    cik_dict = {}
    
    for ticker in tickers:
        URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany&output=atom'.format(ticker)

#         print(URL)


        sec_data = sec_api.get(URL)
        feed = BeautifulSoup(sec_data.encode('ascii'), 'html').feed
        
        print(type(feed))
        
        if feed is not None:
            result = feed.find('cik')

            print(len(result))
            print(result)

            if len(result):

                cik_dict[str(ticker).upper()] = result.get_text()

            print(result.get_text())
    

    return cik_dict

In [7]:
example_ticker = 'AMZN'
sec_data = {}


for ticker, cik in cik_lookup.items():
    sec_data[ticker] = get_sec_data(cik)

pprint.pprint(sec_data[example_ticker][:5])

https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001130310&type=&dateb=&owner=include&count=0&search_text=100&output=atom
[u'2020-08-06', u'2020-08-06', u'2020-08-03', u'2020-07-31', u'2020-07-31', u'2020-07-30', u'2020-07-30', u'2020-07-30', u'2020-07-20', u'2020-07-06']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000093410&type=&dateb=&owner=include&count=0&search_text=100&output=atom
[u'2020-08-06', u'2020-08-05', u'2020-07-31', u'2020-07-31', u'2020-07-31', u'2020-07-22', u'2020-07-20', u'2020-07-20', u'2020-07-20', u'2020-07-20']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000773840&type=&dateb=&owner=include&count=0&search_text=100&output=atom
[u'2020-08-05', u'2020-08-04', u'2020-08-03', u'2020-08-03', u'2020-08-03', u'2020-07-30', u'2020-07-29', u'2020-07-28', u'2020-07-28', u'2020-07-24']
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000034903&type=&dateb=&owner=include&count=0&search_text=100&output=atom

In [76]:
cikamzn = get_sec_CIK_From_Ticker(x)

<class 'bs4.element.Tag'>
1
<cik>0001420529</cik>
0001420529
<class 'bs4.element.Tag'>
1
<cik>0001802457</cik>
0001802457
<class 'bs4.element.Tag'>
1
<cik>0000006201</cik>
0000006201
<class 'bs4.element.Tag'>
1
<cik>0000008177</cik>
0000008177
<class 'bs4.element.Tag'>
1
<cik>0001158114</cik>
0001158114
<class 'bs4.element.Tag'>
1
<cik>0000824142</cik>
0000824142
<class 'bs4.element.Tag'>
1
<cik>0000320193</cik>
0000320193
<class 'bs4.element.Tag'>
1
<cik>0001135185</cik>
0001135185
<class 'bs4.element.Tag'>
1
<cik>0001069183</cik>
0001069183
<class 'bs4.element.Tag'>
1
<cik>0000351569</cik>
0000351569
<class 'bs4.element.Tag'>
1
<cik>0000318306</cik>
0000318306
<class 'bs4.element.Tag'>
1
<cik>0000907654</cik>
0000907654
<class 'bs4.element.Tag'>
1
<cik>0000815094</cik>
0000815094
<class 'bs4.element.Tag'>
1
<cik>0001642081</cik>
0001642081
<class 'bs4.element.Tag'>
1
<cik>0001447028</cik>
0001447028
<class 'bs4.element.Tag'>
1
<cik>0001070494</cik>
0001070494
<class 'bs4.element.Tag'

<class 'bs4.element.Tag'>
1
<cik>0001754068</cik>
0001754068
<class 'bs4.element.Tag'>
1
<cik>0000899866</cik>
0000899866
<class 'bs4.element.Tag'>
1
<cik>0001810182</cik>
0001810182
<class 'bs4.element.Tag'>
1
<cik>0001734520</cik>
0001734520
<class 'bs4.element.Tag'>
1
<cik>0000792977</cik>
0000792977
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0000006951</cik>
0000006951
<class 'bs4.element.Tag'>
1
<cik>0001280263</cik>
0001280263
<class 'bs4.element.Tag'>
1
<cik>0001744494</cik>
0001744494
<class 'bs4.element.Tag'>
1
<cik>0001744494</cik>
0001744494
<class 'bs4.element.Tag'>
1
<cik>0001744494</cik>
0001744494
<class 'bs4.element.Tag'>
1
<cik>0001514991</cik>
0001514991
<class 'bs4.element.Tag'>
1
<cik>0000002488</cik>
0000002488
<class 'bs4.element.Tag'>
1
<cik>0000896262</cik>
0000896262
<class 'bs4.element.Tag'>
1
<cik>0001083446</cik>
0001083446
<class 'bs4.element.Tag'>
1
<cik>0000318154</cik>
0000318154
<class 'bs4.element.Tag'>
1
<cik>0001788028</cik>
0001788028
<class

<class 'bs4.element.Tag'>
1
<cik>0001488039</cik>
0001488039
<class 'bs4.element.Tag'>
1
<cik>0001604464</cik>
0001604464
<class 'bs4.element.Tag'>
1
<cik>0001323885</cik>
0001323885
<class 'bs4.element.Tag'>
1
<cik>0000701288</cik>
0000701288
<class 'bs4.element.Tag'>
1
<cik>0000008063</cik>
0000008063
<class 'bs4.element.Tag'>
1
<cik>0001016169</cik>
0001016169
<class 'bs4.element.Tag'>
1
<cik>0000894081</cik>
0000894081
<class 'bs4.element.Tag'>
1
<cik>0000718877</cik>
0000718877
<class 'bs4.element.Tag'>
1
<cik>0001644963</cik>
0001644963
<class 'bs4.element.Tag'>
1
<cik>0000883948</cik>
0000883948
<class 'bs4.element.Tag'>
1
<cik>0000883948</cik>
0000883948
<class 'bs4.element.Tag'>
1
<cik>0000750574</cik>
0000750574
<class 'bs4.element.Tag'>
1
<cik>0001086434</cik>
0001086434
<class 'bs4.element.Tag'>
1
<cik>0001600620</cik>
0001600620
<class 'bs4.element.Tag'>
1
<cik>0001730463</cik>
0001730463
<class 'bs4.element.Tag'>
1
<cik>0001023364</cik>
0001023364
<class 'bs4.element.Tag'

<class 'bs4.element.Tag'>
1
<cik>0000846546</cik>
0000846546
<class 'bs4.element.Tag'>
1
<cik>0001808898</cik>
0001808898
<class 'bs4.element.Tag'>
1
<cik>0001776985</cik>
0001776985
<class 'bs4.element.Tag'>
1
<cik>0000702513</cik>
0000702513
<class 'bs4.element.Tag'>
1
<cik>0000875357</cik>
0000875357
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001494582</cik>
0001494582
<class 'bs4.element.Tag'>
1
<cik>0000034067</cik>
0000034067
<class 'bs4.element.Tag'>
1
<cik>0001005516</cik>
0001005516
<class 'bs4.element.Tag'>
1
<cik>0001275101</cik>
0001275101
<class 'bs4.element.Tag'>
1
<cik>0001813756</cik>
0001813756
<class 'bs4.element.Tag'>
1
<cik>0001624512</cik>
0001624512
<class 'bs4.element.Tag'>
1
<cik>0000821127</cik>
0000821127
<class 'bs4.element.Tag'>
1
<cik>0001597264</cik>
0001597264
<class 'bs4.element.Tag'>
1
<cik>0000763901</cik>
0000763901
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001133818</cik>
0001133818
<class 'bs4.e

<class 'bs4.element.Tag'>
1
<cik>0001435064</cik>
0001435064
<class 'bs4.element.Tag'>
1
<cik>0001435064</cik>
0001435064
<class 'bs4.element.Tag'>
1
<cik>0001435064</cik>
0001435064
<class 'bs4.element.Tag'>
1
<cik>0001173489</cik>
0001173489
<class 'bs4.element.Tag'>
1
<cik>0001458412</cik>
0001458412
<class 'bs4.element.Tag'>
1
<cik>0001691507</cik>
0001691507
<class 'bs4.element.Tag'>
1
<cik>0001070680</cik>
0001070680
<class 'bs4.element.Tag'>
1
<cik>0001728041</cik>
0001728041
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001728041</cik>
0001728041
<class 'bs4.element.Tag'>
1
<cik>0000913341</cik>
0000913341
<class 'bs4.element.Tag'>
1
<cik>0001490906</cik>
0001490906
<class 'bs4.element.Tag'>
1
<cik>0001305773</cik>
0001305773
<class 'bs4.element.Tag'>
1
<cik>0001478069</cik>
0001478069
<class 'bs4.element.Tag'>
1
<cik>0001527166</cik>
0001527166
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001119774</cik>
0001119774
<class 'bs4.element.Tag'>
1
<cik>0001349929</cik>

...

<class 'bs4.element.Tag'>
1
<cik>0000352960</cik>
0000352960
<class 'bs4.element.Tag'>
1
<cik>0000720005</cik>
0000720005
<class 'bs4.element.Tag'>
1
<cik>0000352960</cik>
0000352960
<class 'bs4.element.Tag'>
1
<cik>0000352960</cik>
0000352960
<class 'bs4.element.Tag'>
1
<cik>0000352960</cik>
0000352960
<class 'bs4.element.Tag'>
1
<cik>0001805284</cik>
0001805284
<class 'bs4.element.Tag'>
1
<cik>0001037038</cik>
0001037038
<class 'bs4.element.Tag'>
1
<cik>0001171155</cik>
0001171155
<class 'bs4.element.Tag'>
1
<cik>0001398987</cik>
0001398987
<class 'bs4.element.Tag'>
1
<cik>0001052595</cik>
0001052595
<class 'bs4.element.Tag'>
1
<cik>0000084246</cik>
0000084246
<class 'bs4.element.Tag'>
1
<cik>0001511337</cik>
0001511337
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001519401</cik>
0001519401
<class 'bs4.element.Tag'>
1
<cik>0001581091</cik>
0001581091
<class 'bs4.element.Tag'>
1
<cik>0000943819</cik>
0000943819
<class 'bs4.element.Tag'>
1
<cik>0001716621</cik>
0001716621
<class

<class 'bs4.element.Tag'>
1
<cik>0001650729</cik>
0001650729
<class 'bs4.element.Tag'>
1
<cik>0000701374</cik>
0000701374
<class 'bs4.element.Tag'>
1
<cik>0000091928</cik>
0000091928
<type 'NoneType'>
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0000091419</cik>
0000091419
<class 'bs4.element.Tag'>
1
<cik>0000932872</cik>
0000932872
<class 'bs4.element.Tag'>
1
<cik>0000319655</cik>
0000319655
<class 'bs4.element.Tag'>
1
<cik>0000766829</cik>
0000766829
<class 'bs4.element.Tag'>
1
<cik>0001015650</cik>
0001015650
<class 'bs4.element.Tag'>
1
<cik>0000899715</cik>
0000899715
<class 'bs4.element.Tag'>
1
<cik>0001065837</cik>
0001065837
<class 'bs4.element.Tag'>
1
<cik>0000090896</cik>
0000090896
<class 'bs4.element.Tag'>
1
<cik>0000087347</cik>
0000087347
<class 'bs4.element.Tag'>
1
<cik>0001524741</cik>
0001524741
<class 'bs4.element.Tag'>
1
<cik>0001097362</cik>
0001097362
<class 'bs4.element.Tag'>
1
<cik>0001040971</cik>
0001040971
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<ci

<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0000024545</cik>
0000024545
<type 'NoneType'>
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0000906338</cik>
0000906338
<class 'bs4.element.Tag'>
1
<cik>0001092289</cik>
0001092289
<class 'bs4.element.Tag'>
1
<cik>0000732717</cik>
0000732717
<class 'bs4.element.Tag'>
1
<cik>0000732717</cik>
0000732717
<class 'bs4.element.Tag'>
1
<cik>0000768899</cik>
0000768899
<class 'bs4.element.Tag'>
1
<cik>0000733590</cik>
0000733590
<class 'bs4.element.Tag'>
1
<cik>0000890319</cik>
0000890319
<type 'NoneType'>
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001075607</cik>
0001075607
<class 'bs4.element.Tag'>
1
<cik>0001411688</cik>
0001411688
<class 'bs4.element.Tag'>
1
<cik>0000947263</cik>
0000947263
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0000816761</cik>
0000816761
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0000919893</cik>
0000919893
<class 'bs4.element.Tag'>
1
<cik>0001260221</cik>
0001260221
<type 'NoneType'>
<t

<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001418091</cik>
0001418091
<class 'bs4.element.Tag'>
1
<cik>0001342874</cik>
0001342874
<class 'bs4.element.Tag'>
1
<cik>0000217346</cik>
0000217346
<type 'NoneType'>
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001268533</cik>
0001268533
<class 'bs4.element.Tag'>
1
<cik>0000860731</cik>
0000860731
<class 'bs4.element.Tag'>
1
<cik>0001336917</cik>
0001336917
<class 'bs4.element.Tag'>
1
<cik>0001336917</cik>
0001336917
<class 'bs4.element.Tag'>
1
<cik>0001114446</cik>
0001114446
<class 'bs4.element.Tag'>
1
<cik>0000101538</cik>
0000101538
<class 'bs4.element.Tag'>
1
<cik>0001425292</cik>
0001425292
<class 'bs4.element.Tag'>
1
<cik>0000831001</cik>
0000831001
<class 'bs4.element.Tag'>
1
<cik>0000008504</cik>
0000008504
<class 'bs4.element.Tag'>
1
<cik>0001029800</cik>
0001029800
<class 'bs4.element.Tag'>
1
<cik>0001543151</cik>
0001543151
<class 'bs4.element.

<class 'bs4.element.Tag'>
1
<cik>0000740260</cik>
0000740260
<class 'bs4.element.Tag'>
1
<cik>0000884219</cik>
0000884219
<class 'bs4.element.Tag'>
1
<cik>0001713952</cik>
0001713952
<type 'NoneType'>
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001674910</cik>
0001674910
<class 'bs4.element.Tag'>
1
<cik>0000312070</cik>
0000312070
<class 'bs4.element.Tag'>
1
<cik>0000312070</cik>
0000312070
<class 'bs4.element.Tag'>
1
<cik>0000732712</cik>
0000732712
<class 'bs4.element.Tag'>
1
<cik>0001616707</cik>
0001616707
<class 'bs4.element.Tag'>
1
<cik>0000943452</cik>
0000943452
<class 'bs4.element.Tag'>
1
<cik>0001212545</cik>
0001212545
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001000697</cik>
0001000697
<class 'bs4.element.Tag'>
1
<cik>0001517496</cik>
0001517496
<class 'bs4.element.Tag'>
1
<cik>0000719245</cik>
0000719245
<class 'bs4.element.Tag'>
1
<cik>0000801337</cik>
0000801337
<type 'NoneType'>
<class 'bs4.element.Tag'>
1
<cik>0001650962</cik>
0001650962
<class 'bs4.e

In [82]:
# len(cikamzn)

len(cikamzn.values())

5762

In [83]:
import json
app_json = json.dumps(cikamzn)
print(cikamzn)

with open('ticksvsciks2.json', 'w') as json_file:
    json.dump(cikamzn, json_file)
    
    

{'LASR': u'0001124796', 'SPR': u'0001364885', 'SPT': u'0001517375', 'SPH': u'0001005210', 'SPI': u'0001210618', 'SPN': u'0000886835', 'SPB': u'0000109177', 'SPE': u'0000897802', 'SPG': u'0001063761', 'XPO': u'0001166003', 'XPL': u'0000917225', 'PQG': u'0001708035', 'REML': u'0001053092', 'OKTA': u'0001660134', 'JNJ': u'0000200406', 'TRHC': u'0001651561', 'BEN': u'0000038777', 'BCYC': u'0001761612', 'SLDB': u'0001707502', 'BEP': u'0001533232', 'GRU': u'0000352960', 'VNTR': u'0001705682', 'GRX': u'0001391437', 'GRF': u'0000850027', 'GRC': u'0000042682', 'GRA': u'0001045309', 'GRN': u'0000312070', 'LPSN': u'0001102993', 'ITGR': u'0001114483', 'NOG': u'0001104485', 'NOA': u'0001368519', 'NOC': u'0001133421', 'HLI': u'0001302215', 'DYAI': u'0001213809', 'HLG': u'0001596964', 'HLF': u'0001180262', 'NOV': u'0001021860', 'NOW': u'0001373715', 'ATKR': u'0001666138', 'OLLI': u'0001639300', 'SYPR': u'0000864240', 'WSC': u'0001647088', 'BWXT': u'0001486957', 'FXNC': u'0000719402', 'ENZ': u'0000316

In [8]:
Counter(sec_data['AMZN'])



Counter({u'2020-07-02': 2,
         u'2020-07-30': 1,
         u'2020-07-31': 2,
         u'2020-08-05': 5})

In [21]:
sec_data

{'AMZN': [u'2020-08-05',
  u'2020-08-05',
  u'2020-08-05',
  u'2020-08-05',
  u'2020-08-05',
  u'2020-07-31',
  u'2020-07-31',
  u'2020-07-30',
  u'2020-07-02',
  u'2020-07-02'],
 'BMY': [u'2020-08-06',
  u'2020-08-06',
  u'2020-08-04',
  u'2020-07-30',
  u'2020-07-16',
  u'2020-07-16',
  u'2020-07-13',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-02'],
 'CNP': [u'2020-08-06',
  u'2020-08-06',
  u'2020-08-03',
  u'2020-07-31',
  u'2020-07-31',
  u'2020-07-30',
  u'2020-07-30',
  u'2020-07-30',
  u'2020-07-20',
  u'2020-07-06'],
 'CVX': [u'2020-08-06',
  u'2020-08-05',
  u'2020-07-31',
  u'2020-07-31',
  u'2020-07-31',
  u'2020-07-22',
  u'2020-07-20',
  u'2020-07-20',
  u'2020-07-20',
  u'2020-07-20'],
 'FL': [u'2020-08-03',
  u'2020-08-03',
  u'2020-07-23',
  u'2020-07-16',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-06',
  u'2020-07-06'],
 'FRT': [u'2020-08-05', u'2020-08-05'],
 'HON': [u'2020-08-05',
  u'2020-08-04',
  u'2020-08-03',
  u'2020-08-

In [13]:


DEFAULT_TICKERS = ['goog', 'aapl']
URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
CIK_RE = re.compile(r'.*CIK=(\d{10}).*')

cik_dict = {}
for ticker in DEFAULT_TICKERS:
    results = CIK_RE.findall(get(URL.format(ticker)).content)
    if len(results):
        cik_dict[str(ticker).lower()] = str(results[0])
f = open('cik_dict', 'w')
dump(cik_dict, f)
f.close()

In [47]:
raw_fillings_by_ticker = {}

for ticker, data in sec_data.items():
    raw_fillings_by_ticker[ticker] = {}
    for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
        if (file_type == '10-K'):
            file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            
            raw_fillings_by_ticker[ticker][file_date] = sec_api.get(file_url)


print('Example Document:\n\n{}...'.format(next(iter(raw_fillings_by_ticker[example_ticker].values()))[:1000]))

Example Document:

-----BEGIN PRIVACY-ENHANCED MESSAGE-----
Proc-Type: 2001,MIC-CLEAR
Originator-Name: webmaster@www.sec.gov
Originator-Key-Asymmetric:
 MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen
 TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB
MIC-Info: RSA-MD5,RSA,
 JheN9rrH0LADcPLNpdgic2Y+wa34llXqe750TowVzUFLd6IuuNQne2H0axAosYFz
 1Lmg9dJfEMxBnv2E6NCBrw==

<SEC-DOCUMENT>0001193125-05-047032.txt : 20050311
<SEC-HEADER>0001193125-05-047032.hdr.sgml : 20050311
<ACCEPTANCE-DATETIME>20050311062027
ACCESSION NUMBER:		0001193125-05-047032
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		9
CONFORMED PERIOD OF REPORT:	20041231
FILED AS OF DATE:		20050311
DATE AS OF CHANGE:		20050311

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			AMAZON COM INC
		CENTRAL INDEX KEY:			0001018724
		STANDARD INDUSTRIAL CLASSIFICATION:	RETAIL-CATALOG & MAIL-ORDER HOUSES [5961]
		IRS NUMBER:				911646860
		STATE OF INCORPORATION:			DE
		FISCAL YEAR END:			1231

	FILING VA

In [3]:


DEFAULT_TICKERS = ['goog', 'aapl','amzn']
URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
CIK_RE = re.compile(r'.*CIK=(\d{10}).*')

cik_dict = {}
for ticker in DEFAULT_TICKERS:
    results = CIK_RE.findall(get(URL.format(ticker)).content)
    if len(results):
        cik_dict[str(ticker).lower()] = str(results[0])
f = open('cik_dict', 'w')
dump(cik_dict, f)
f.close()

In [28]:

def getCIKs(TICKERS):
    URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    CIK_RE = re.compile(r'.*CIK=(\d{10}).*')    
    cik_dict = {}
    for ticker in TICKERS:
        print(URL.format(ticker))
        f = sec_api.get(URL.format(ticker))
        
#         f = requests.get(URL.format(ticker), stream = True)
        results = CIK_RE.findall(f.text)
        print(cik_dict)
    
        if len(results):
            results[0] = int(re.sub('\.[0]*', '.', results[0]))
            cik_dict[str(ticker).upper()] = str(results[0])
    
    print(cik_dict)
    

getCIKs(['wmt','amzn','nflx'])

http://www.sec.gov/cgi-bin/browse-edgar?CIK=wmt&Find=Search&owner=exclude&action=getcompany


AttributeError: 'unicode' object has no attribute 'text'

In [27]:
y = getCIKs(x)

AttributeError: 'unicode' object has no attribute 'text'

In [25]:
y

NameError: name 'y' is not defined

In [13]:
def StockList():
        StocksController = Stocklist.NasdaqController(True)
        list_of_tickers = StocksController.getList()

        print(list_of_tickers)
        print(len(list_of_tickers))
        return list_of_tickers

In [58]:
x = StockList()

['AACG', 'AACQU', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPL', 'AAWW', 'AAXN', 'ABCB', 'ABEO', 'ABIO', 'ABMD', 'ABTX', 'ABUS', 'ACAD', 'ACAM', 'ACAMU', 'ACAMW', 'ACBI', 'ACCD', 'ACER', 'ACEVU', 'ACGL', 'ACGLO', 'ACGLP', 'ACHC', 'ACHV', 'ACIA', 'ACIU', 'ACIW', 'ACLS', 'ACMR', 'ACNB', 'ACOR', 'ACRS', 'ACRX', 'ACST', 'ACTG', 'ADAP', 'ADBE', 'ADES', 'ADI', 'ADIL', 'ADILW', 'ADMA', 'ADMP', 'ADMS', 'ADP', 'ADPT', 'ADRO', 'ADSK', 'ADTN', 'ADTX', 'ADUS', 'ADVM', 'ADXN', 'ADXS', 'AEGN', 'AEHR', 'AEIS', 'AEMD', 'AERI', 'AESE', 'AEY', 'AEYE', 'AEZS', 'AFH', 'AFIB', 'AFIN', 'AFINP', 'AFMD', 'AFYA', 'AGBA', 'AGBAR', 'AGBAU', 'AGBAW', 'AGEN', 'AGFS', 'AGIO', 'AGLE', 'AGMH', 'AGNC', 'AGNCM', 'AGNCN', 'AGNCO', 'AGNCP', 'AGRX', 'AGTC', 'AGYS', 'AHCO', 'AHPI', 'AIH', 'AIHS', 'AIKI', 'AIMC', 'AIMT', 'AINV', 'AIRG', 'AIRT', 'AIRTP', 'AIRTW', 'AKAM', 'AKBA', 'AKCA', 'AKER', 'AKRO', 'AKTS', 'AKTX', 'AKUS', 'ALAC', 'ALACR', 'ALACU', 'ALACW', 'ALBO', 'ALCO', 'ALDX', 'ALEC', 'ALGN', 'ALGT', 'ALIM', 'ALJJ', 'ALKS', '

In [59]:
len(set(x))

6750

In [17]:
index = pd.date_range(start='2020-07-01', end='today')

In [18]:
index

DatetimeIndex(['2020-07-01', '2020-07-02', '2020-07-03', '2020-07-04',
               '2020-07-05', '2020-07-06', '2020-07-07', '2020-07-08',
               '2020-07-09', '2020-07-10', '2020-07-11', '2020-07-12',
               '2020-07-13', '2020-07-14', '2020-07-15', '2020-07-16',
               '2020-07-17', '2020-07-18', '2020-07-19', '2020-07-20',
               '2020-07-21', '2020-07-22', '2020-07-23', '2020-07-24',
               '2020-07-25', '2020-07-26', '2020-07-27', '2020-07-28',
               '2020-07-29', '2020-07-30', '2020-07-31', '2020-08-01',
               '2020-08-02', '2020-08-03', '2020-08-04', '2020-08-05',
               '2020-08-06', '2020-08-07', '2020-08-08', '2020-08-09'],
              dtype='datetime64[ns]', freq='D')

In [19]:
df = pd.DataFrame(0 , index = index, columns=x)

In [20]:
df.tail()

,AACG,AACQU,AAL,AAME,AAOI,AAON,AAPL,AAWW,AAXN,ABCB,...,ZNH,ZOM,ZTEST,ZTO,ZTR,ZTS,ZUO,ZVV,ZXIET,ZYME
2020-08-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-08-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-08-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-08-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-08-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
